<a href="https://colab.research.google.com/github/dorinhazan/FinalProject-DataScience/blob/main/classification_grouping_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import openai
import zipfile
import json
import os
from striprtf.striprtf import rtf_to_text

In [36]:
# !pip install openai
# !pip install --upgrade openai
# !pip install striprtf --quiet

In [37]:
# 1) Load the RTF
with open("token_api_final_proj.rtf", "r", encoding="utf-8") as f:
    rtf = f.read()

key = rtf_to_text(rtf).strip()
os.environ["OPENAI_API_KEY"] = key

print("OPENAI_API_KEY loaded:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY loaded: True


In [32]:
results_ranking_path = "merged_ranking.txt"
ranking_res_zip = "/content/ranking_results.zip"

with zipfile.ZipFile(ranking_res_zip, 'r') as z, open(results_ranking_path, 'w', encoding='utf-8') as out:
    for info in z.infolist():
        if info.filename.endswith('.txt'):
            out.write(f"===== {info.filename} =====\n")
            with z.open(info) as f:
                for raw in f:
                    out.write(raw.decode('utf-8', errors='replace'))
            out.write("\n\n")

print(f"Done — all .txt merged into `{results_ranking_path}`")


FileNotFoundError: [Errno 2] No such file or directory: '/content/ranking_results.zip'

In [ ]:
obs_res_zip        = '/content/json_results.zip'
extract_dir     = '/content/extracted_jsons'
merged_json_path = 'merged_obs_extraction.json'

with zipfile.ZipFile(obs_res_zip, 'r') as zf:
    zf.extractall(extract_dir)

# 3. Merge into one dictionary, extending duplicates’ observables
merged_report_results = {}
for filename in os.listdir(extract_dir):
    # only process .json files
    if not filename.lower().endswith('.json'):
        continue

    full_path = os.path.join(extract_dir, filename)
    with open(full_path, 'r') as f:
        data = json.load(f)

    for md_name, doc in data.items():
        if md_name not in merged_report_results:
            merged_report_results[md_name] = doc
        else:
            # extend observables list from both
            merged_report_results[md_name]["observables"].extend(doc.get("observables", []))
            # you can merge other fields here if needed

with open(merged_json_path, 'w') as f:
    json.dump(merged_report_results, f, indent=2)

Create Classification Groups

In [48]:
merged_json_path = 'merged_obs_extraction.json'
with open(merged_json_path, 'r', encoding='utf-8') as f:
    merged_obs_extraction = json.load(f)

In [49]:
# Extract, lowercase, and dedupe classification values
unique_classifications = {
    obs['classification'].lower()
    for doc in merged_obs_extraction.values()
    for obs in doc.get('observables', [])
    if 'classification' in obs
}

In [50]:
unique_classifications.update(['api',
 'api call',
 'api function',
 'api hook',
 'archive file',
 'attack effect',
 'attack technique',
 'authentication credential',
 'authentication technique',
 'boot record',
 'c2 server',
 'circuit breaker',
 'code snippet',
 'command',
 'command line argument',
 'command message',
 'command parameters',
 'command sequence',
 'command value',
 'command/utility',
 'communication channel',
 'communication interface',
 'communication port',
 'communication priority level',
 'communication protocol',
 'config/data file',
 'configuration data',
 'configuration file',
 'configuration parameter',
 'credential',
 'credential list',
 'data',
 'data block',
 'data manipulation',
 'data transfer',
 'database server',
 'device',
 'device configuration value',
 'device state',
 'device status parameter',
 'distributed control system',
 'dll file',
 'document',
 'domain name',
 'driver file',
 'dynamic-link library',
 'electrical device',
 'email message',
 'error state',
 'event log',
 'executable command',
 'executable file',
 'exploit payload',
 'exploit technique',
 'facility',
 'file',
 'file type',
 'file/script',
 'firewall rule',
 'firewall utility',
 'firmware',
 'frequency converter drive',
 'function block memory location',
 'hardware component',
 'hex value',
 'hmi device',
 'host device',
 'http request',
 'ics address',
 'ics application',
 'ics command',
 'ics command message',
 'ics communication payload',
 'ics communication protocol',
 'ics configuration',
 'ics configuration file',
 'ics controller',
 'ics data',
 'ics data block recording',
 'ics device',
 'ics device parameter',
 'ics field',
 'ics function block',
 'ics gateway',
 'ics i/o snapshot',
 'ics network communication',
 'ics object',
 'ics parameter',
 'ics payload component',
 'ics process data',
 'ics project file',
 'ics protocol',
 'ics protocol field',
 'ics protocol module',
 'ics protocol parameter',
 'ics protocol value',
 'ics software module',
 'ics systems',
 'ics tag',
 'identifier constant',
 'impact condition',
 'industrial communication protocol',
 'industrial control device',
 'industrial control module',
 'industrial control system',
 'industrial control system (ics) device',
 'industrial control system device',
 'industrial control system platform',
 'industrial controller (cpu)',
 'industrial network bus',
 'integrity check routine',
 'log files',
 'malicious file',
 'malicious ics function block',
 'malicious logic',
 'malicious software',
 'malware',
 'malware module',
 'manufacturing facility',
 'medical product',
 'memory access pattern',
 'memory access request',
 'memory write attempt',
 'module',
 'monitor',
 'network communication',
 'network communication technique',
 'network configuration',
 'network connection',
 'network data',
 'network device',
 'network entity',
 'network identifier',
 'network packet',
 'network port',
 'network protocol',
 'network segment',
 'network service',
 'network service/port',
 'network traffic',
 'network traffic event',
 'opc server',
 'opc ua address space',
 'opc ua server',
 'operating system api',
 'operational mode',
 'os system call',
 'packaging tool',
 'payload file',
 'plc',
 'plc block',
 'plc code',
 'plc code component',
 'plc cpu',
 'plc data block',
 'plc data block content (magic value)',
 'plc device',
 'plc function block',
 'plc i/o image',
 'plc operating system',
 'plc organization block',
 'plc program organization unit',
 'plc runtime constraint',
 'process',
 'process name list',
 'process termination',
 'profibus communication module',
 'program transfer operation',
 'project file',
 'protective relay',
 'protocol',
 'protocol field',
 'protocol object data',
 'protocol version',
 'proxy server',
 'python function call',
 'registry key',
 'remote access service',
 'remote terminal unit',
 'removable media',
 'resource identifier',
 'safety instrumented system',
 'script',
 'script engine/interpreter',
 'script file',
 'script framework',
 'sensor reading',
 'serial port',
 'server',
 'server information',
 'service control',
 'service/process',
 'servo drive',
 'shellcode',
 'signature value',
 'software/program',
 'software/tool',
 'software/tool (malware)',
 'sql command',
 'sql stored procedure',
 'storage device',
 'string',
 'url',
 'user account',
 'variable state',
 'vulnerability',
 'vulnerability identifier',
 'web script',
 'windows registry key'])

In [51]:
gross_classifications = set([
"System",
"Module",
"Hardware",
"Device",
"PLC",
"POU",
"Identifier",
"Information",
"Credential",
"Data",
"Command",
"API",
"Code",
"Process",
"Configuration",
"Log",
"File",
"Payload",
"Software",
"Service",
"Firmware",
"Server",
"Communication",
"Protocol",
"Protocol Field",
"Connection Port",
"Vulnerability",
"Attack Technique"])

In [52]:
classification_grouping_prompt = f"""
You are a cybersecurity analyst expert. Your job is to group a list of observable “classification” labels under gross classifications from MITRE ATT&CK® Enterprise, or if not matched to any existing gross classification - create a new category that matches the observable group.

---
### Input
   - A set of unique observable classifications:
     {sorted(unique_classifications)}

   - A set of gross classifications from MITRE ATT&CK® Enterprise:
     {sorted(gross_classifications)}

---
### Task
   - For each unique observable classifications, choose exactly one gross classification it best fits.
   - If it does not clearly belong to any of the gross classification labels given, assign it to a new category of your own choosing, appending “– new” to that category name (e.g. “Data Management – new”).

---
### Output
   - Return **only** a single JSON object.
   - Do **not** include any words outside of that JSON—no explanations, no commentary.
   - Do **not** add free text; only JSON format.
   - Each key must be either one of the 28 gross classification label names or one of your “– new” categories.
   - Each value must be an array of the unique observable classifications assigned to that key.

---
### Response format)

```json
{{
  "Code": [
    "script file"
  ],
  "Command": [
    "command string"
  ],
  "Identifier": [
    "ip address",
    "domain"
  ],
  "Data": [
    "file format",
    "file signature"
  ]
  ...
}}

##

"""

In [53]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],  # or however you load it
)


response = client.chat.completions.create(
    model="o3-mini-2025-01-31",          # ← swap in your desired model
    messages=[
        {"role": "system", "content": "You are a helpful cybersecurity expert."},
        {"role": "user",   "content": classification_grouping_prompt},
    ],
)

# Extract the raw JSON string from the assistant’s reply
raw = response.choices[0].message.content


In [54]:
# Strip out any leading/trailing whitespace
clean_response = raw.strip()

# We look for the first `{` and the last `}`
start = clean_response.find('{')
end   = clean_response.rfind('}')
if start != -1 and end != -1:
    clean_response = clean_response[start:end+1]

groups_mapping = json.loads(clean_response)
with open("groups_mapping.json", "w") as f:
    json.dump(groups_mapping, f, indent=2)

print("groups_mapping.json written")

groups_mapping.json written


In [55]:
# Build reverse map: sub-classification (lowercase) → gross group name
reverse_map = {}
for gross_group, sub_list in groups_mapping.items():
    for sub in sub_list:
        reverse_map[sub.lower()] = gross_group


Rename column and extract only the relavent for comprison

In [56]:
# output_path = 'merged_obs_extraction_renamed.json'

# Iterate over each file and its observables, renaming keys as requested
for filename, content in merged_obs_extraction.items():
    for obs in content.get('observables', []):
        if 'observable_rank' in obs:
            obs['artifact_details'] = obs.pop('observable_rank')
        if 'classification' in obs:
            obs['fine_classification'] = obs.pop('classification')

# # Save the updated JSON to a new file
# with open(output_path, 'w') as f:
#     json.dump(merged_obs_extraction, f, indent=2)

# print(f"Renamed keys and saved the updated JSON to '{output_path}'")

In [57]:
# merged_obs_extraction_renamed_path = "/content/merged_obs_extraction_renamed.json"
# with open(merged_obs_extraction_renamed_path, 'r') as f:
#     merged_obs_extraction_renamed = json.load(f)

In [59]:

# 4) Annotate each observable
for doc_name, doc_data in merged_obs_extraction.items():
    for obs in doc_data.get('observables', []):
        # print(doc_data.get('observables', []))
        cls = obs.get('fine_classification', '').lower()
        # print(cls)
        # look up the group, default to 'Unknown' if not found
        obs['gross_classification'] = reverse_map.get(cls, 'Unknown')

# 5) Save the enriched JSON
out_path = '/content/merged_with_gross_classification-reports.json'
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(merged_obs_extraction, f, indent=2, ensure_ascii=False)

print(f"Done—saved annotated file to {out_path}")

Done—saved annotated file to /content/merged_with_gross_classification-reports.json


Convert csv table into json object in order to compare between observables and add `gross_classification` col

In [72]:
csv_path = "/content/updated_Procedure_descriptions_observables_using_o3_mini_prompt_with_deprecation.csv"
observables_porc_desc_df = pd.read_csv(csv_path)

observables_porc_desc_df = observables_porc_desc_df.rename(columns={"classification": "fine_classification"})

wanted = [
    "observable_value",
    "artifact_details",
    "data_source",
    "fine_classification",
    "notes",
    "description",
    "STIX_supported",
    "proprietary_artifact",
    "parser",
]

# Reindex (fills any missing with None)
observables_porc_desc_df = observables_porc_desc_df.reindex(columns=wanted, fill_value=None)

# Drop duplicates on observable_value
observables_porc_desc_df = observables_porc_desc_df.drop_duplicates(subset=['observable_value'], keep='first')

# Add gross_classification via vectorized map
observables_porc_desc_df['gross_classification'] = (
    observables_porc_desc_df['fine_classification']
      .str.lower()
      .map(reverse_map)      # make sure you’ve defined reverse_map earlier
      .fillna('Unknown')
)

observables_porc_desc_df.set_index("observable_value") \
    .to_json(
       "observables_by_value.json",
       orient="index",
       indent=2,
       force_ascii=False
    )

out_path = '/content/merged_with_gross_classification-table.json'
records = observables_porc_desc_df.to_dict(orient='records')
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved annotated file to {out_path}")


Saved annotated file to /content/merged_with_gross_classification-table.json
